# Modelos de Regresión y Clasificación I
Actividad Lección 3 || Fundamentos de IA y Machine Learning

Objetivos:
* Aplicar conceptos teóricos vistos en clase

Datos del alumno:
* Víctor Luque Martín
* Máster Avanzado en Programación en Python para Hacking, BigData y Machine Learning

Fecha: 20/10/2022

# Tabla de contenidos:
1. [Problema I](#problema-i)
    1. [Preparación Previa](#pi-preparacion-previa)
        1. [Obtención de train y test](#pi-obtencion-train-test)
        2. [Normalización de los datos de entrenamiento](#pi-pp-norm-train)
        3. [Normalicación de los datos de test](#pi-pp-norm-test)
        4. [Obtención del vector de medias del entrenamiento](#pi-pp-media-train)
        5. [Obtención de la matriz de covarianzas del entrenamiento](#pi-pp-cov-train)
    2. [Regresión Lineal Simple](#pi-regresion-lineal-simple)
        1. [Regresión lineal simple Y en función de X<sub>1</sub>](#pi-rls-modelo-1)
        2. [Regresión lineal simple Y en función de X<sub>2</sub>](#pi-rls-modelo-2)
    3. [Regresión lineal múltiple](#pi-regresion-lineal-multiple)
        1. [Operaciones auxiliares](#pi-rlm-operaciones-auxiliares)
        2. [Regresión lineal múltiple de Y en función de X1 y X2](#pi-modelo-rlm)
    4. [Validación de los modelos](#pi-validacion-modelos)
2. [Problema II](#problema-ii)
    1. [Transformación Logística](#pii-transformacion-logistica)
    2. [Evaluación de la regresión logística](#pii-evaluacion-regresion-logistica)
    3. [Comparación con el modelo de la lección 3](#pii-comparacion-modelo-leccion-3)
3. [Problema III](#problema-iii)
    1. [Preparacion previa](#piii-preparacion-previa)
    2. [Algoritmo KNN](#piii-algoritmo-knn)
        1. [Resultados k = 1](#piii-resultados-k-1)
        2. [Resultados k = 3](#piii-resultados-k-3)
    3. [Conclusiones](#piii-conclusiones)

In [1]:
import pandas as pd

# Problema I <a class="anchor" id="problema-i"></a>
Se desea estimar el valor Y de una casa dada su superficie X<sub>1</sub> y el número de habitaciones X<sub>2</sub> . Hasta ahora, se conocen los siguientes datos de entrenamiento:

In [2]:
train_p1 = pd.read_csv('l3p1_train.csv')
train_p1

,x1,x2,y
0,60,1,120
1,55,2,125
2,70,1,130
3,60,2,112
4,62,1,122
5,70,2,135
6,60,3,128
7,80,1,140
8,80,2,150
9,85,3,160


Se pide hallar:
1. El modelo de regresión lineal simple de Y en función de X<sub>1</sub> .
2. El modelo de regresión lineal simple de Y en función de X<sub>2</sub> .
3. El modelo de regresión lineal múltiple de Y en función de X<sub>1</sub> y X<sub>2</sub> .
4. Validar cada modelo con el siguiente conjunto de test.

In [3]:
test_p1 = pd.read_csv('l3p1_test.csv')
test_p1

,x1,x2,y
0,58,1,120
1,50,3,128
2,80,1,140
3,65,2,115
4,55,1,120
5,75,2,145
6,55,3,125
7,80,3,170
8,70,1,135
9,75,3,155


## Preparación previa <a class="anchor" id="pi-preparacion-previa"></a>
Antes de comenzar a resolver el problema se deben realizar las siguientes tareas:
1. Normalizar los datos de entrenamiento.
2. Normalizar los datos de test.
3. Obtención del vector de medias del entrenamiento.
4. Obtención de la matriz de covarianzas del entrenamiento.

### Obtención de train y test <a class="anchor" id="pi-obtencion-train-test"></a>

### Normalización de los datos de entrenamiento <a class="anchor" id="pi-pp-norm-train"></a>
Para normalizar los datos de entrenamiento se debe realizar la siguiente operación:

$$X_{normalized} = \frac{X - X_{min}}{X_{max} - X_{min}}$$

In [4]:
# Normalización Min-Max Train
df = train_p1.drop('y', axis=1)
p1_max = df.max()
p1_min = df.min()
df_norm = (df-p1_min)/(p1_max-p1_min)
train_p1_norm = pd.concat((train_p1.y, df_norm), axis=1)
train_p1_norm.head()

,y,x1,x2
0,120,0.166667,0.0
1,125,0.000000,0.5
2,130,0.500000,0.0
3,112,0.166667,0.5
4,122,0.233333,0.0


### Normalización de los datos de test <a class="anchor" id="pi-pp-norm-test"></a>
Para normalizar los datos de test se deben utilizar los mismos valores de $X_{min}$ y $X_{max}$ que se utilizaron para normalizar los datos de entrenamiento. Se expresaría de la siguiente forma:

$$X'_{normalized} = \frac{X' - X_{min}}{X_{max} - X_{min}}$$

In [5]:
# Normalización Min-Max Test
df = test_p1.drop('y', axis=1)
df_norm = (df-p1_min)/(p1_max-p1_min)
test_p1_norm = pd.concat((test_p1.y, df_norm), axis=1)
test_p1_norm.head()

,y,x1,x2
0,120,0.100000,0.0
1,128,-0.166667,1.0
2,140,0.833333,0.0
3,115,0.333333,0.5
4,120,0.000000,0.0


### Obtención del vector de medias del entrenamiento <a class="anchor" id="pi-pp-media-train"></a>
Para obtener el vector de medias del entrenamiento se debe obtener la media de cada columna de la matriz de datos de entrenamiento. Se expresaría de la siguiente forma:

$$\mu = \frac{1}{m} \sum_{i=1}^{m} x^{(i)}$$

Utilizaremos la función mean() de pandas para calcular la media de cada columna.

In [6]:
vector_medias = train_p1_norm.mean()
vector_medias

y     132.20
x1      0.44
x2      0.40
dtype: float64

### Obtención de la matriz de covarianzas del entrenamiento <a class="anchor" id="pi-pp-cov-train"></a>
Para obtener la matriz de covarianzas del entrenamiento se debe obtener la covarianza de cada columna de la matriz de datos de entrenamiento. Se expresaría de la siguiente forma:

$$\Sigma_{ij} = \text{Cov}(X_i, X_j)$$

Si $i = j$ entonces:

$$\Sigma_{ii} = \text{Var}(X_i)$$

Utilizaremos la función [cov()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.cov.html) de pandas para obtener la matriz de covarianzas del entrenamiento.

In [7]:
matriz_covarianza = train_p1_norm.cov()
matriz_covarianza

,y,x1,x2
y,210.400000,4.550370,2.244444
x1,4.550370,0.121185,0.017407
x2,2.244444,0.017407,0.155556


## Regresión lineal simple <a class="anchor" id="pi-regresion-lineal-simple"></a>
El modelo de regresión a estimar es $Y = \hat{\beta_0} + \hat{\beta_1} X$. Para obtener los parámetros $\hat{\beta_0}$ y $\hat{\beta_1}$ se debe resolver el siguiente sistema de ecuaciones:

$$\text{Pendiente} = \hat{\beta_1} = \frac{S_{x,y}}{S^2_x}$$

$$\text{Ordenada de Origen} = \hat{\beta_0} = \mu_y - \hat{\beta_1} \mu_x = \bar{Y} - \hat{\beta_1} \bar{X_1}$$

Siendo $S$ la matriz de covarianzas del entrenamiento y $\mu$ el vector de medias del entrenamiento.

Ambas ecuaciones vienen definidas dentro de la siguiente función:

In [8]:
def regresion_lineal_simple(train, y, avg_y, x, avg_x):
    # Calculamos la matriz de covarianzas
    matriz_covarianzas = train.cov() 
    # obtener la varianza de x
    varianza_x = matriz_covarianzas.loc[x, x]
    # obtener la covarianza entre x e y
    covarianza_x_y = matriz_covarianzas.loc[x, y]
    # obtener la varianza de y
    varianza_y = matriz_covarianzas.loc[y, y]
    # obtener la beta 1
    beta_1 = covarianza_x_y / varianza_x
    # obtener el termino independiente
    beta_0 = avg_y - beta_1 * avg_x
    # obtener la ecuacion de la recta
    sub = str.maketrans("0123456789", "₀₁₂₃₄₅₆₇₈₉")
    ecuacion = f"Y = {beta_0} + {beta_1}·{x.translate(sub)}"

    # Almacenar los resultados en un diccionario
    resultados = {
        "beta_1": beta_1,
        "beta_0": beta_0,
        "ecuacion": ecuacion,
        "data": []
    }
    # Calcular los valores de Y para cada valor de X
    for row in train.iterrows():
        resultados["data"].append(
            beta_0 + beta_1 * row[1][x]
        )
    resultados["data"] = pd.Series(resultados["data"])        
    return resultados

### Regresión lineal simple Y en función de X<sub>1</sub> <a class="anchor" id="pi-rls-modelo-1"></a>
Aplicamos la función anterior para obtener los resultados del modelo 1.

In [9]:
p1_modelo_1 = regresion_lineal_simple(train_p1_norm, 'y', vector_medias[0], 'x1', vector_medias[1])

**El valor de $\hat{\beta_1}$ es:**
$$\hat{\beta_1} = 37.548899755501225$$

In [10]:
p1_modelo_1["beta_1"]

37.548899755501225

**El valor de $\hat{\beta_0}$ es:**
$$\hat{\beta_0} = 115.67848410757945$$

In [11]:
p1_modelo_1["beta_0"]

115.67848410757945

**El modelo estimado sería:**<br>
$$Y = 115.67848410757945 + 37.548899755501225·X_1$$

In [12]:
p1_modelo_1["ecuacion"]

'Y = 115.67848410757945 + 37.548899755501225·x₁'

### Regresión lineal simple Y en función de X<sub>2</sub> <a class="anchor" id="pi-rls-modelo-2"></a>

In [13]:
p1_modelo_2 = regresion_lineal_simple(train_p1_norm, 'y', vector_medias[0], 'x2', vector_medias[2])

**El valor de $\hat{\beta_1}$ es:**
$$\hat{\beta_1} = 14.428571428571429$$

In [14]:
p1_modelo_2["beta_1"]

14.428571428571429

**El valor de $\hat{\beta_0}$ es:**
$$\hat{\beta_0} = 126.42857142857142$$

In [15]:
p1_modelo_2["beta_0"]

126.42857142857142

**El modelo estimado sería:**<br>
$$Y = 126.42857142857142 + 14.428571428571429·X_2$$

In [16]:
p1_modelo_2["ecuacion"]

'Y = 126.42857142857142 + 14.428571428571429·x₂'

## Regresión lineal múltiple <a class="anchor" id="pi-regresion-lineal-multiple"></a>
El modelo de regresión a estimar es $Y = \hat{\beta_0} + \hat{\beta_1} X_1 + \hat{\beta_2} X_2$. Para obtener los parámetros $\hat{\beta_0}$, $\hat{\beta_1}$ y $\hat{\beta_2}$ se debe resolver el siguiente sistema de ecuaciones:

$$\text{Coeficientes Parciales} = \hat{\beta_i} = - \frac{A_{1, j+1}}{A_{1, 1}} \rightarrow \begin{cases} \hat{\beta_1} = - \frac{A_{1, 2}}{A_{1, 1}} \\ \hat{\beta_2} = - \frac{A_{1, 3}}{A_{1, 1}} \end{cases}$$

$$\text{Ordenada de Origen} = \hat{\beta_0} = \mu_y - \hat{\beta_1} \mu_{x_1} - \hat{\beta_2} \mu_{x_2} = \bar{Y} - \hat{\beta_1} \bar{X_1} - \hat{\beta_2} \bar{X_2}$$

Siendo $A_{i,j}$ el adjunto ($Adj_{ij}(S) = (-1)^{i+j}\cdot det(S_{ij})$) de la matriz de covarianzas del entrenamiento y $\mu$ el vector de medias del entrenamiento.

### Operaciones auxiliares <a class="anchor" id="pi-rlm-operaciones-auxiliares"></a>
Para resolver el sistema de ecuaciones anterior se necesitan realizar las siguientes operaciones:
1. Calcular el adjunto de la matriz de covarianzas 3x3 del entrenamiento.
2. Obtener los coeficientes parciales $\hat{\beta_1}$ y $\hat{\beta_2}$.
3. Obtener la ordenada de origen $\hat{\beta_0}$.

**Adjunto de la matriz de covarianzas 3x3 del entrenamiento**<br>
A partir de la siguiente función se obtiene el adjunto de la matriz de covarianzas 3x3 del entrenamiento, dado un índice de fila y columna.

In [17]:
def calc_adjunto_matriz_3x3(matriz_covarianzas, col, idx):
    # Eliminar la columna y la fila de la matriz de covarianzas
    # para obtener la matriz adjunta
    matriz_adjunta = matriz_covarianzas.copy()\
        .drop(matriz_covarianzas.index[idx-1], axis=0)\
        .drop(matriz_covarianzas.columns[col-1], axis=1)

    # Calcular el determinante de la matriz adjunta
    det = matriz_adjunta.iloc[0, 0] * matriz_adjunta.iloc[1, 1] - matriz_adjunta.iloc[0, 1] * matriz_adjunta.iloc[1, 0]
    # Si la suma de idx y col es impar el adjunto es negativo
    adjunto = (-1)**(idx+col) * det
    return adjunto

**Coeficientes parciales $\hat{\beta_1}$ y $\hat{\beta_2}$**<br>
A partir de la siguiente función se obtienen los coeficientes parciales $\hat{\beta_1}$ y $\hat{\beta_2}$ en formato de lista, siendo el primer elemento $\hat{\beta_1}$ y el segundo $\hat{\beta_2}$.

In [18]:
def coeficientes_parciales(matriz_covarianzas):
    # Calcular el denominador A11
    denominador = calc_adjunto_matriz_3x3(matriz_covarianzas, col=1, idx=1)
    coeficientes = []
    # Recorremos el indice de las columnas menos la primera
    for col in range(2, len(matriz_covarianzas)+1):
        # Calcular el numerador A1i+1
        numerador = calc_adjunto_matriz_3x3(matriz_covarianzas, col=col, idx=1)
        # Calcular el coeficiente parcial (- (A1i+1 / A11))
        c = - (numerador / denominador)
        # Agregar el coeficiente parcial a la lista
        coeficientes.append(c)
    return coeficientes

**Ordenada de origen $\hat{\beta_0}$**<br>
A partir de la siguiente función se obtiene la ordenada de origen $\hat{\beta_0}$.

In [19]:
def ordenada_origen(coeficientes, vector_medias):
    # Ordenada de origen b0 = media_y - b1·media_x1 - b2·media_x2
    # Inicialmente b0 = y
    ordenada_origen = vector_medias[0]
    # Recorremos los coeficientes parciales
    for idx, coeficiente in enumerate(coeficientes):
        # En cada iteración restamos bi·media_xi
        # El idx comienza en 0, por lo que le sumamos 1
        ordenada_origen -= coeficiente * vector_medias[idx+1]
    return ordenada_origen

### Regresión lineal múltiple de Y en función de X1 y X2 <a class="anchor" id="pi-modelo-rlm"></a>
Aplicamos las funciones anteriores para obtener del modelo 3.

In [20]:
def regresion_lineal_multiple(matriz, vector_medias):
    # Calcular la matriz de covarianzas
    matriz_covarianzas = matriz.cov()
    # Calcular los coeficientes parciales
    coeficientes = coeficientes_parciales(matriz_covarianzas)
    # Calcular la ordenada de origen
    beta_0 = ordenada_origen(coeficientes, vector_medias)
    sub = str.maketrans("0123456789", "₀₁₂₃₄₅₆₇₈₉")
    # Calcular la ecuación
    ecuacion = f"y = {beta_0} + {coeficientes[0]}·{'x1'.translate(sub)} + {coeficientes[1]}·{'x2'.translate(sub)}"
    resultados = {
        "coeficientes": coeficientes,
        "beta_0": beta_0,
        "ecuacion": ecuacion,
        "data": []
    }
    # Calcular los valores de Y para cada valor de X1 y X2
    for row in matriz.iterrows():
        y = beta_0
        for idx, coeficiente in enumerate(coeficientes):
            y += coeficiente * row[1][idx+1]
        resultados["data"].append(y)
    resultados["data"] = pd.Series(resultados["data"])
    return resultados

In [21]:
p1_modelo_3 = regresion_lineal_multiple(train_p1_norm, vector_medias)

**El valor de $\hat{\beta_1}$ es:**
$$\hat{\beta_1} = 36.055910956624636$$

In [22]:
p1_modelo_3["coeficientes"][0]

36.055910956624636

**El valor de $\hat{\beta_2}$ es:**
$$\hat{\beta_2} = 10.393743297711053$$

In [23]:
p1_modelo_3["coeficientes"][1]

10.393743297711053

**El valor de $\hat{\beta_0}$ es:**
$$\hat{\beta_0} = 112.17790186000072$$

In [24]:
p1_modelo_3["beta_0"]

112.17790186000072

**El modelo estimado sería:**<br>
$$Y = 112.17790186000072 + 36.055910956624636·X_1 + 10.393743297711053·X_2$$

In [25]:
p1_modelo_3["ecuacion"]

'y = 112.17790186000072 + 36.055910956624636·x₁ + 10.393743297711053·x₂'

## Validación de los modelos <a class="anchor" id="pi-validacion-modelos"></a>
Esta vez se utilizará el conjunto de datos de test para validar los modelos estimados. Posteriormente se hallarán las métricas de error para cada modelo.

Los modelos a validar son los siguientes:
1. Modelo 1: Regresión lineal simple de Y en función de X1. 

$$Y = 115.67848410757945 + 37.548899755501225·X_1$$

2. Modelo 2: Regresión lineal simple de Y en función de X2. 

$$Y = 126.42857142857142 + 14.428571428571429·X_2$$

3. Modelo 3: Regresión lineal múltiple de Y en función de X1 y X2. 

$$Y = 112.17790186000072 + 36.055910956624636·X_1 + 10.393743297711053·X_2$$

Los resultados serían los siguientes:

In [26]:
data_pred = {"Y": test_p1_norm["y"],"Ym1": [],"Ym2": [],"Ym3": []}
for row in test_p1_norm.iterrows():
    data_pred["Ym1"].append(p1_modelo_1["beta_0"] + p1_modelo_1["beta_1"] * row[1]["x1"])
    data_pred["Ym2"].append(p1_modelo_2["beta_0"] + p1_modelo_2["beta_1"] * row[1]["x2"])
    data_pred["Ym3"].append(p1_modelo_3["beta_0"] + p1_modelo_3["coeficientes"][0] *\
        row[1]["x1"] + p1_modelo_3["coeficientes"][1] * row[1]["x2"])

df_pred_test = pd.DataFrame(data_pred)
df_pred_test

,Y,Ym1,Ym2,Ym3
0,120,119.433374,126.428571,115.783493
1,128,109.420334,140.857143,116.562327
2,140,146.969234,126.428571,142.224494
3,115,128.194784,133.642857,129.393410
4,120,115.678484,126.428571,112.177902
5,145,140.711084,133.642857,141.412047
6,125,115.678484,140.857143,122.571645
7,170,146.969234,140.857143,152.618238
8,135,134.452934,126.428571,130.205857
9,155,140.711084,140.857143,146.608919


Para cada modelo se calculan los residuos 

$$e_i = y_i - \hat{y_i}$$

In [27]:
# Para cada Ym calcular los residuos e = Y_train_pred - Y_test_pred
data = {"em1": [],"em2": [],"em3": []}
for row in df_pred_test.iterrows():
    data["em1"].append(test_p1_norm["y"][row[0]] - row[1]["Ym1"])
    data["em2"].append(test_p1_norm["y"][row[0]] - row[1]["Ym2"])
    data["em3"].append(test_p1_norm["y"][row[0]] - row[1]["Ym3"])

df_residuos = pd.DataFrame(data)
df_residuos

,em1,em2,em3
0,0.566626,-6.428571,4.216507
1,18.579666,-12.857143,11.437673
2,-6.969234,13.571429,-2.224494
3,-13.194784,-18.642857,-14.393410
4,4.321516,-6.428571,7.822098
5,4.288916,11.357143,3.587953
6,9.321516,-15.857143,2.428355
7,23.030766,29.142857,17.381762
8,0.547066,8.571429,4.794143
9,14.288916,14.142857,8.391081


**Reutilizamos las funciones para calcular las métricas de regresión y las encapsulamos en una clase.**

In [28]:
class MetricasRegresion:
    def __init__(self, y, y_pred):
        self.y = y
        self.y_pred = y_pred
        self.n = len(y)

    @property
    def mae(self):
        sumatorio = 0
        for y, m in zip(self.y, self.y_pred):
            sumatorio += abs(y - m)
        return sumatorio / self.n

    @property
    def mse(self):
        sumatorio = 0
        for y, m in zip(self.y, self.y_pred):
            sumatorio += (y - m) ** 2
        return sumatorio / self.n

    @property
    def rmse(self):
        return self.mse ** 0.5

    @property
    def r2(self):
        media_y = sum(self.y) / self.n
        numerador, denominador = 0, 0
        for y, m in zip(self.y, self.y_pred):
            numerador += (m - media_y) ** 2
            denominador += (y - media_y) ** 2
        return numerador / denominador

**Calculamos las métricas de error para cada modelo.**

In [29]:
# calcular las métricas para cada modelo
mr_1 = MetricasRegresion(test_p1_norm["y"], df_pred_test["Ym1"])
mr_2 = MetricasRegresion(test_p1_norm["y"], df_pred_test["Ym2"])
mr_3 = MetricasRegresion(test_p1_norm["y"], df_pred_test["Ym3"])
data = [
    [mr_1.mae, mr_1.mse, mr_1.rmse, mr_1.r2],
    [mr_2.mae, mr_2.mse, mr_2.rmse, mr_2.r2],
    [mr_3.mae, mr_3.mse, mr_3.rmse, mr_3.r2]
]
sup = str.maketrans("0123456789", "⁰¹²³⁴⁵⁶⁷⁸⁹")
pd.DataFrame(
    data, 
    columns=["MAE", "MSE", "RMSE", "R2".translate(sup)], 
    index=["Modelo 1", "Modelo 2", "Modelo 3"]
)

,MAE,MSE,RMSE,R²
Modelo 1,9.510901,142.704714,11.945908,0.754564
Modelo 2,13.700000,228.292857,15.109363,0.161504
Modelo 3,7.667748,83.619318,9.144360,0.728719


**Conclusiones**<br>
- Todas las métricas indican que el mejor modelo es el 3, seguido del 1 y por último el 2.
- Observamos que la regresión lineal múltiple es mejor que la regresión lineal simple, ya que tiene un menor error cuadrático medio y un menor error absoluto medio, es necesario el uso de $X_1$ y $X_2$ para obtener un mejor modelo.
- La $\beta_1$ tiene un valor mayor que $\beta_2$, por lo que $X_1$ es más importante que $X_2$ para explicar la variable dependiente $Y$.
- El error con mayor puntuación es el error cuadrático medio, en los tres modelos.

# Problema II <a class="anchor" id="problema-ii"></a>
Haciendo uso de la base de datos Iris vista anteriormente, se ha ejecutado un algoritmo para estimar los parámetros de una regresión logística múltiple. Como el problema tiene tres clases: setosa (1), versicolor (2) y virgínica (3), se han generado los dos modelos siguientes:

$$f_1(X,\beta_1) = 541.0741 – 33.121X_1 − 10.2824X_2 − 38.5734X_3 − 90.4374X_4$$

$$f_2(X,\beta_2) = 501.1562 − 26.3497X_1 − 24.2689X_2 − 10.7613X_3 − 130.0915X_4$$

Se pide:
1. Hallar las predicciones del modelo para el siguiente conjunto de test.
2. Evaluar el rendimiento del clasificador en dicho conjunto.
3. Comparar el modelo obtenido con el expuesto en la lección 3.

Almacenamos en una lista el valor de cada $f_i$.

In [30]:
# [beta_0, beta_1, beta_2, beta_3, beta_4]
p2_f1 = [541.0741, 33.121, 10.2824, 38.5734, 90.4374]
p2_f2 = [501.1562, 26.3497, 24.2689, 10.7613, 130.0915]
p2_df = pd.read_csv("l3p2_test.csv")
p2_df

,x1,x2,x3,x4,clase
0,4.6,3.2,1.4,0.2,1
1,5.3,3.7,1.5,0.2,1
2,5.7,4.4,1.5,0.4,1
3,5.0,3.5,1.6,0.6,2
4,5.5,2.5,4.0,1.3,2
5,5.7,3.0,4.2,1.2,2
6,5.7,2.8,4.1,1.3,3
7,5.8,2.7,5.1,1.9,3
8,6.3,2.5,5.0,1.9,3
9,5.9,3.0,5.1,1.8,3


## Transformación logística <a class="anchor" id="pii-transformacion-logistica"></a>
Para cada patrón se deben obtener los siguientes resultados:
$$e^{f_1(X,\beta_1)}$$

$$e^{f_2(X,\beta_2)}$$

$$\sum_{i=1}^{n} e^{f_i(X,\beta_i)}$$

$$p(Y=q|X) = \frac{e^{\beta_0 + \sum_{i=0}^{n} \beta_{i}·X_{i}}}{1 + e^{\beta_0 + \sum_{l=1}^{Q-1} \beta_{lk}·X_{lk}}}$$

- Clase Predicha
- Clase Real

Para poder trabajar con el número $e$ en Python, se debe importar la librería math.

In [31]:
import math
# Empleado para hacer prints en Jupyter
from IPython.display import display

**Creación de la función para calcular el valor de $e^{f_i(X,\beta_i)}$**

In [32]:
def e_f(data, fi):
    lista_efi = []
    for row in data.iterrows():
        y = fi[0]
        for idx, beta in enumerate(fi[1:]):
            y -= beta * row[1][idx]
        lista_efi.append(math.exp(y)) # math.e ** y
    return lista_efi

**Creación de la función para calcular el valor de $\sum_{i=1}^{n} e^{f_i(X,\beta_i)}$**

In [33]:
def sum_ef(ef1, ef2):
    return [e_1 + e_2 for e_1, e_2 in zip(ef1, ef2)]

**Creación de la función para calcular el valor de $p(Y=q|X)$**

In [34]:
def probabilidad(ef, sum_ef):
    return [e / (1+s) for e, s in zip(ef, sum_ef)]

**Creación dataframe con los resultados**

In [35]:
p2_e_f1 = e_f(p2_df, p2_f1)
p2_e_f2 = e_f(p2_df, p2_f2)
p2_suma_ef = sum_ef(p2_e_f1, p2_e_f2)
p2_data_stats = {"e^f1": p2_e_f1, "e^f2": p2_e_f2, "SUM(e^f1, e^f2)": p2_suma_ef,
    "P1": probabilidad(p2_e_f1, p2_suma_ef),"P2": probabilidad(p2_e_f2, p2_suma_ef)}
p2_df_stats = pd.DataFrame(p2_data_stats)
# P3 = 1 - P1 - P2
p2_df_stats["P3"] = 1 - p2_df_stats["P1"] - p2_df_stats["P2"]
# La clase predicha debe ser un entero
p2_df_stats["Clase Predicha"] = p2_df_stats[["P1", "P2", "P3"]].idxmax(axis=1).str[-1].astype(int)
p2_df_stats["Clase real"] = p2_df["clase"]
p2_df_stats

,e^f1,e^f2,"SUM(e^f1, e^f2)",P1,P2,P3,Clase Predicha,Clase real
0,1.658002e+123,2.747976e+113,1.658002e+123,1.000000e+00,1.657401e-10,6.337916e-17,1,1
1,1.748162e+109,4.912023e+99,1.748162e+109,1.000000e+00,2.809821e-10,9.549555e-17,1,1
2,3.218616e+92,2.731178e+76,3.218616e+92,1.000000e+00,8.485566e-17,-8.485566e-17,1,1
3,1.161911e+97,1.464615e+82,1.161911e+97,1.000000e+00,1.260523e-15,7.174503e-17,1,2
4,4.363034e+26,1.654314e+36,1.654314e+36,2.637367e-10,1.000000e+00,-1.110223e-16,2,2
5,1.280394e+22,2.371783e+33,2.371783e+33,5.398446e-12,1.000000e+00,0.000000e+00,2,2
6,5.598049e+20,1.998021e+30,1.998021e+30,2.801797e-10,1.000000e+00,1.110223e-16,2,3
7,2.742053e-21,4.343567e-09,4.343567e-09,2.742053e-21,4.343567e-09,1.000000e+00,3,3
8,6.520509e-26,3.100338e-12,3.100338e-12,6.520509e-26,3.100338e-12,1.000000e+00,3,3
9,3.869177e-20,9.579455e-08,9.579455e-08,3.869177e-20,9.579454e-08,9.999999e-01,3,3


## Evaluación de la regresión logística <a class="anchor" id="pii-evaluacion-regresion-logistica"></a>
Para evaluar el rendimiento del clasificador se calculan las siguientes métricas:
1. Matriz de confusión
2. CCR
3. Kappa
4. TPR
5. FPR
6. TNR
7. PPV
8. F1-Score

**Reutilizamos las funciones para calcular las métricas de problemas de clasificación utilizadas en la anterior lección y las encapsulamos en una clase.**<br>
Para calcular la matriz de confusión usamos el método pd.crosstab().

In [36]:
class MetricasClasificacion:
    def __init__(self, y, y_pred) -> None:
        self.y = y
        self.y_pred = y_pred
        self.n = len(y)
        self.matriz_confusion = pd.crosstab(self.y, self.y_pred, rownames=["Real"], colnames=["Predicha"])

    def crear_sub_matriz(self, c):
        matriz = self.matriz_confusion.copy()
        matriz["Negativo"] = matriz.sum(axis=1) - matriz[c]
        matriz["Positivo"] = matriz[c]
        matriz = matriz[["Positivo", "Negativo"]].rename(index={c: "Positivo"})
        matriz.loc["Negativo"] = matriz.sum(axis=0) - matriz.loc["Positivo"]
        matriz = matriz.drop(matriz.index.difference(["Positivo", "Negativo"]))
        return matriz

    def ccr(self):
        confusion = self.matriz_confusion.copy()
        columnas = confusion.columns
        filas = confusion.index
        suma_matriz = confusion.sum().sum()
        numerador = 0
        for c, f in zip(columnas, filas):
            if columnas.get_loc(c) == filas.get_loc(f):
                numerador += confusion[c][f]
        return numerador / suma_matriz
            

    def tpr(self, c = None):
        try:
            if c is not None:
                confusion = self.crear_sub_matriz(c)
            else:
                confusion = self.matriz_confusion.copy()
            return confusion["Positivo"]["Positivo"] / \
                (confusion["Positivo"]["Positivo"] + confusion["Negativo"]["Positivo"])
        except Exception:
            return 0

    def fpr(self, c = None):
        try:    
            if c is not None:
                confusion = self.crear_sub_matriz(c)
            else:
                confusion = self.matriz_confusion.copy()
            return confusion["Positivo"]["Negativo"] / \
                (confusion["Negativo"]["Negativo"] + confusion["Positivo"]["Negativo"])
        except Exception:
            return 0

    def tnr(self, c = None):
        try:
            if c is not None:
                confusion = self.crear_sub_matriz(c)
            else:
                confusion = self.matriz_confusion.copy()
            return confusion["Negativo"]["Negativo"] / \
                (confusion["Negativo"]["Negativo"] + confusion["Positivo"]["Negativo"])
        except Exception:
            return 0

    def ppv(self, c = None):
        try:
            if c is not None:
                confusion = self.crear_sub_matriz(c)
            else:
                confusion = self.matriz_confusion.copy()
            return confusion["Positivo"]["Positivo"] / \
                (confusion["Positivo"]["Positivo"] + confusion["Positivo"]["Negativo"])
        except Exception:
            return 0

    def f1(self, c = None):
        try:
            return 2 * (self.ppv(c) * self.tpr(c)) / \
                (self.ppv(c) + self.tpr(c))
        except Exception:
            return 0

    def kappa(self):
        confusion = self.matriz_confusion.copy()
        confusion.loc["Total"] = confusion.sum()
        confusion["Total"] = confusion.sum(axis=1)
        total_col = confusion["Total"]
        total_row = confusion.loc["Total"]
        total = confusion.loc["Total", "Total"]
        total_row = total_row.drop("Total")
        total_col = total_col.drop("Total")
        pe = 0
        for c, f in zip(total_col.index, total_row.index):
            pe += (total_col[c] * total_row[f])/(total*total)
        k = (self.ccr() - pe) / (1 - pe)
        return k

**Matriz de confusión**

In [37]:
p2_metricas_clasificacion = MetricasClasificacion(p2_df_stats["Clase real"], p2_df_stats["Clase Predicha"])
p2_metricas_clasificacion.matriz_confusion

Predicha,1,2,3
Real,,,
1,3,0,0
2,1,2,0
3,0,1,3


**CCR y Kappa**

In [38]:
print(f"CCR: {p2_metricas_clasificacion.ccr()}")
print(f"Kappa: {p2_metricas_clasificacion.kappa()}")

CCR: 0.8
Kappa: 0.701492537313433


**Creación de matriz de confusión para cada clase**

In [39]:
for i in range(1, 4):
    print("=============================")
    print(f"Matriz Confusión - Clase {i}")
    display(p2_metricas_clasificacion.crear_sub_matriz(i))

Matriz Confusión - Clase 1


Predicha,Positivo,Negativo
Real,,
Positivo,3,0
Negativo,1,6


Matriz Confusión - Clase 2


Predicha,Positivo,Negativo
Real,,
Positivo,2,1
Negativo,1,6


Matriz Confusión - Clase 3


Predicha,Positivo,Negativo
Real,,
Positivo,3,1
Negativo,0,6


**Métricas para cada clase**

In [40]:
p2_metricas_data = [
    {
        "Sensibilidad": p2_metricas_clasificacion.tpr(i), "False Positive Rate": p2_metricas_clasificacion.fpr(i),
        "Especificidad": p2_metricas_clasificacion.tnr(i), "Precision": p2_metricas_clasificacion.ppv(i),
        "F1 Score": p2_metricas_clasificacion.f1(i)
    } for i in range(1, 4)
]
p2_df_metricas = pd.DataFrame(p2_metricas_data, index=["1", "2", "3"])
p2_df_metricas.index.name = "Clase"
p2_df_metricas.loc["Promedio"] = p2_df_metricas.mean()
p2_df_metricas = p2_df_metricas.style.apply(
    lambda x: ["background: yellow" if x.name == "Promedio" else "" for i in x], axis=1)
p2_df_metricas

,Sensibilidad,False Positive Rate,Especificidad,Precision,F1 Score
Clase,,,,,
1,1.000000,0.142857,0.857143,0.750000,0.857143
2,0.666667,0.142857,0.857143,0.666667,0.666667
3,0.750000,0.000000,1.000000,1.000000,0.857143
Promedio,0.805556,0.095238,0.904762,0.805556,0.793651


## Comparación con el modelo de la lección 3 <a class="anchor" id="pii-comparacion-modelo-leccion-3"></a>
Tras comparar los resultados obtenidos con el modelo de la lección 3, podemos observar lo siguiente:
- La matriz de confusión es la misma.
- El CCR tiene el mismo porcentaje (80% - 80%).
- El Kappa es aproximadamente el mismo (0.70 - 0.701492537313433).
- Las métricas de cada matriz de confusion para cada clase son similares, en caso de realizar un aproximación a tres decimales.
- El promedio de las métricas de cada matriz de confusión es similar, en caso de realizar un aproximación a tres decimales.

Con esto podemos concluir que el modelo obtenido es similar o practicamente igual al modelo de la lección 3.

# Problema III <a class="anchor" id="problema-iii"></a>
Para un problema de clasificación binaria y con tres características se han recogido los siguientes patrones de entrenamiento

In [41]:
p3_df_train = pd.read_csv("l3p3_train.csv")
p3_df_train

,x1,x2,x3,clase
0,4.6,3.2,1.4,1
1,5.3,3.7,1.5,3
2,5.7,4.4,1.5,1
3,5.0,3.5,1.6,2
4,5.5,2.5,4.0,1
5,5.7,3.0,4.2,2
6,5.7,2.8,4.1,2
7,5.8,2.7,5.1,1
8,6.3,2.5,5.0,2
9,5.9,3.0,5.1,3


Se pide:
1. Hallar las predicciones del modelo para el siguiente conjunto de test, con el valor de 𝑘 = 1 y 𝑘 = 3 vecinos, y la distancia euclídea.
2. Evaluar el rendimiento del clasificador en dicho conjunto para ambos valores de 𝑘.

In [42]:
p3_df_test = pd.read_csv("l3p3_test.csv")
p3_df_test

,x1,x2,x3,clase
0,5.0,3.5,1.7,1
1,4.3,2.8,1.5,1
2,2.7,4.5,1.2,3
3,5.0,4.2,1.3,3
4,6.3,2.5,4.1,1
5,5.2,3.0,4.5,2
6,4.5,3.0,4.2,2
7,5.9,2.9,5.2,2
8,5.0,2.4,5.1,1
9,4.5,3.2,5.0,2


## Preparacion previa <a class="anchor" id="piii-preparacion-previa"></a>
Antes de resolver el problema es necesario realizar una serie de preparaciones previas:
1. Definir la distancia euclídea como métrica de distancia.

**Distancia Euclídea**<br>
Es la raíz cuadrada de las diferencias al cuadrado de cada una de las coordenadas de los puntos.

$$d_E(X_1,X_2) = \sqrt{\sum_{i=1}^{k} (X_{1i} - X_{2i})^2}$$

In [43]:
def distancia_euclidea(x1, x2):
    if len(x1) != len(x2):
        raise Exception("Las coordenadas deben tener la misma cantidad de elementos")
    k = len(x1)
    sumatorio = sum([(x1[i] - x2[i])**2 for i in range(k)])
    return sumatorio**0.5

## Algoritmo KNN <a class="anchor" id="piii-algoritmo-knn"></a>
Calculamos la distancia de cada patrón de prueba a los de entrenamiento, se seleccionan los tres más cercanos y se determina la clase predicha

In [44]:
# algoritmo knn
def knn(train, test, k):
    train, test = train.copy(), test.copy()
    data = {}
    # Iterar sobre cada fila del dataframe de test
    for i, row in test.iterrows():
            distancias = []
            # Iterar sobre cada fila del dataframe de entrenamiento
            for j, row_train in train.iterrows():
                # Calcular la distancia euclidea entre la fila de test y la fila de train
                distancia = distancia_euclidea(row_train[:-1], row[:-1])
                # Agregar la columna del train j y la fila de test i al df
                distancias.append(distancia)
            # Agregar la columna de distancias al diccionario
            data[i] = distancias

    # Crear un dataframe con las distancias
    df_distancias = pd.DataFrame(data, columns=[i for i, row in test.iterrows()])
    df_distancias.index.name = "P. Test"
    df_distancias.columns.name = "Patrón Entrenamiento"

    predicted = []
    # Recorrer las distancias
    for i, row in df_distancias.iterrows():
        tmp = []
        # Convertir las distancias a una lista y ordenarlas de menor a mayor
        # Obtener los k vecinos más cercanos y recorrerlos
        for m in sorted(row.tolist())[:k]:
            # Obtener la columna el indice columna de train y
            # obtener la clase de la fila de train
            tmp.append(train.iloc[row[row == m].index[0]]["clase"])
        # Obtener la clase más frecuente
        predicted.append(max(set(tmp), key = tmp.count))

    df_distancias["Clase Predicha"] = predicted
    df_distancias["Clase Real"] = test["clase"]

    return df_distancias

### Resultados k = 1 <a class="anchor" id="piii-resultados-k-1"></a>

In [45]:
p3_knn_d1 = knn(p3_df_train, p3_df_test, k=1)
p3_knn_d1

Patrón Entrenamiento,0,1,2,3,4,5,6,7,8,9,Clase Predicha,Clase Real
P. Test,,,,,,,,,,,,
0,0.583095,0.509902,2.310844,1.081665,3.266497,3.163858,2.808914,4.027406,3.806573,3.601389,3.0,1
1,0.412311,1.345362,2.736786,0.616441,3.033150,3.082207,2.901724,3.832754,3.839271,3.624914,1.0,1
2,1.157584,2.126029,3.016621,0.754983,3.275668,3.348134,3.269557,3.997499,4.177320,3.889730,2.0,3
3,0.100000,0.994987,2.539685,0.761577,2.989983,2.949576,2.694439,3.758989,3.668787,3.449638,1.0,3
4,2.557342,2.789265,4.436215,3.229551,0.806226,0.768115,1.135782,1.326650,1.212436,1.577973,2.0,1
5,2.643861,3.047950,4.500000,3.215587,0.787401,0.583095,1.200000,1.024695,1.288410,1.456022,2.0,2
6,2.596151,2.952965,4.505552,3.207803,0.670820,0.670820,1.220656,1.122497,1.284523,1.552417,1.0,2
7,3.583295,3.901282,5.297169,4.162932,1.135782,0.900000,1.609348,0.244949,0.854400,1.396424,1.0,2
8,3.685105,4.042277,5.603570,4.274342,0.900000,1.307670,2.032240,0.600000,1.307670,1.931321,1.0,1


**Matriz de confusión para 𝑘 = 1**

In [46]:
p3_metricas_knn_d1 = MetricasClasificacion(p3_knn_d1["Clase Real"], p3_knn_d1["Clase Predicha"])
p3_metricas_knn_d1.matriz_confusion

Predicha,1.0,2.0,3.0
Real,,,
1,2,1,1
2,3,1,0
3,1,1,0


**Métricas para 𝑘 = 1**

In [47]:
p3_metricas_knn_d1_data = [
    {
        "Sensibilidad": p3_metricas_knn_d1.tpr(i), "False Positive Rate": p3_metricas_knn_d1.fpr(i),
        "Especificidad": p3_metricas_knn_d1.tnr(i), "Precision": p3_metricas_knn_d1.ppv(i),
        "F1 Score": p3_metricas_knn_d1.f1(i)
    } for i in range(1, 4)
]
p3_df_metricas_knn_d1_details = pd.DataFrame(p3_metricas_knn_d1_data, index=["1", "2", "3"])
p3_df_metricas_knn_d1_details.index.name = "Clase"
p3_df_metricas_knn_d1_details = p3_df_metricas_knn_d1_details.fillna(0)
p3_df_metricas_knn_d1_details.loc["Promedio"] = p3_df_metricas_knn_d1_details.mean()
p3_df_metricas_knn_d1_details = p3_df_metricas_knn_d1_details.style.apply(
    lambda x: ["background: yellow" if x.name == "Promedio" else "" for i in x], axis=1)
print(f"Métricas de clasificación para k = 1")
print(f"CCR: {p3_metricas_knn_d1.ccr()}")
print(f"Kappa: {p3_metricas_knn_d1.kappa()}")
p3_df_metricas_knn_d1_details

Métricas de clasificación para k = 1
CCR: 0.3
Kappa: -0.12903225806451615


C:\Users\Usuario\AppData\Local\Temp\ipykernel_24044\1243272186.py:75: RuntimeWarning: invalid value encountered in double_scalars
  return 2 * (self.ppv(c) * self.tpr(c)) / \


,Sensibilidad,False Positive Rate,Especificidad,Precision,F1 Score
Clase,,,,,
1,0.500000,0.666667,0.333333,0.333333,0.400000
2,0.250000,0.333333,0.666667,0.333333,0.285714
3,0.000000,0.125000,0.875000,0.000000,0.000000
Promedio,0.250000,0.375000,0.625000,0.222222,0.228571


### Resultados k = 3 <a class="anchor" id="piii-resultados-k-3"></a>

In [48]:
p3_knn_d3 = knn(p3_df_train, p3_df_test, k=3)
p3_knn_d3

Patrón Entrenamiento,0,1,2,3,4,5,6,7,8,9,Clase Predicha,Clase Real
P. Test,,,,,,,,,,,,
0,0.583095,0.509902,2.310844,1.081665,3.266497,3.163858,2.808914,4.027406,3.806573,3.601389,1.0,1
1,0.412311,1.345362,2.736786,0.616441,3.033150,3.082207,2.901724,3.832754,3.839271,3.624914,1.0,1
2,1.157584,2.126029,3.016621,0.754983,3.275668,3.348134,3.269557,3.997499,4.177320,3.889730,1.0,3
3,0.100000,0.994987,2.539685,0.761577,2.989983,2.949576,2.694439,3.758989,3.668787,3.449638,1.0,3
4,2.557342,2.789265,4.436215,3.229551,0.806226,0.768115,1.135782,1.326650,1.212436,1.577973,2.0,1
5,2.643861,3.047950,4.500000,3.215587,0.787401,0.583095,1.200000,1.024695,1.288410,1.456022,1.0,2
6,2.596151,2.952965,4.505552,3.207803,0.670820,0.670820,1.220656,1.122497,1.284523,1.552417,1.0,2
7,3.583295,3.901282,5.297169,4.162932,1.135782,0.900000,1.609348,0.244949,0.854400,1.396424,2.0,2
8,3.685105,4.042277,5.603570,4.274342,0.900000,1.307670,2.032240,0.600000,1.307670,1.931321,1.0,1


**Matriz de confusión para 𝑘 = 3**<br>
Es necesario realizar una pequeña corrección en la matriz de confusión, ya que no tenemos resultados predichos para la clase 3.

In [49]:
p3_metricas_knn_d3 = MetricasClasificacion(p3_knn_d3["Clase Real"], p3_knn_d3["Clase Predicha"])
p3_metricas_knn_d3.matriz_confusion[3.0] = 0
p3_metricas_knn_d3.matriz_confusion

Predicha,1.0,2.0,3.0
Real,,,
1,3,1,0
2,2,2,0
3,2,0,0


**Métricas para 𝑘 = 3**

In [50]:
p3_metricas_knn_d3_data = [
    {
        "Sensibilidad": p3_metricas_knn_d3.tpr(i), "False Positive Rate": p3_metricas_knn_d3.fpr(i),
        "Especificidad": p3_metricas_knn_d3.tnr(i), "Precision": p3_metricas_knn_d3.ppv(i),
        "F1 Score": p3_metricas_knn_d3.f1(i)
    } for i in range(1, 4)
]
p3_df_metricas_knn_d3_details = pd.DataFrame(p3_metricas_knn_d3_data, index=["1", "2", "3"])
p3_df_metricas_knn_d3_details.index.name = "Clase"
p3_df_metricas_knn_d3_details = p3_df_metricas_knn_d3_details.fillna(0)
p3_df_metricas_knn_d3_details.loc["Promedio"] = p3_df_metricas_knn_d3_details.mean()
p3_df_metricas_knn_d3_details = p3_df_metricas_knn_d3_details.style.apply(
    lambda x: ["background: yellow" if x.name == "Promedio" else "" for i in x], axis=1)
print(f"Métricas de clasificación para k = 1")
print(f"CCR: {p3_metricas_knn_d3.ccr()}")
print(f"Kappa: {p3_metricas_knn_d3.kappa()}")
p3_df_metricas_knn_d3_details

Métricas de clasificación para k = 1
CCR: 0.5
Kappa: 0.16666666666666663


C:\Users\Usuario\AppData\Local\Temp\ipykernel_24044\1243272186.py:68: RuntimeWarning: invalid value encountered in longlong_scalars
  return confusion["Positivo"]["Positivo"] / \
C:\Users\Usuario\AppData\Local\Temp\ipykernel_24044\1243272186.py:68: RuntimeWarning: invalid value encountered in longlong_scalars
  return confusion["Positivo"]["Positivo"] / \
C:\Users\Usuario\AppData\Local\Temp\ipykernel_24044\1243272186.py:68: RuntimeWarning: invalid value encountered in longlong_scalars
  return confusion["Positivo"]["Positivo"] / \


,Sensibilidad,False Positive Rate,Especificidad,Precision,F1 Score
Clase,,,,,
1,0.750000,0.666667,0.333333,0.428571,0.545455
2,0.500000,0.166667,0.833333,0.666667,0.571429
3,0.000000,0.000000,1.000000,0.000000,0.000000
Promedio,0.416667,0.277778,0.722222,0.365079,0.372294


## Conclusiones <a class="anchor" id="piii-conclusiones"></a>
Tras comparar los resultados obtenidos con el modelo de la lección 3, podemos observar lo siguiente:
- Para la distancia k = 1, el CCR nos da un resultado muy bajo (0.3) y el Kappa es negativo, indicando que hay discordancia entre las clases predichas y las reales.
- Para la distancia k = 3, el CCR es superior al anterior (0.5) y el Kappa es positivo pero muy cercano a 0, indicando que hay poca concordancia entre las clases predichas y las reales.
- Las métricas de cada matriz para k = 1, dan resultados muy bajos, indicando que el modelo no es bueno.
- Las métricas de cada matriz para k = 3, dan resultados bajos, algo mejores que el caso anterior.
- Los modelos obtenidos no son buenos, ya que no se ajustan a los datos de entrenamiento.